In [2]:
import os 
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [3]:
import torch
print("Torch version:", torch.__version__)
# GPU Settings
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda:0" if USE_CUDA else "cpu")
NGPU = torch.cuda.device_count()
print('gpu 개수:', NGPU)

Torch version: 2.4.1+cu121
gpu 개수: 1


In [4]:
import timm
import time
import tqdm
import cv2
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import copy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


/home/thaivq/miniconda3/envs/foundation-retina/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from utils.model import ResNetModel

In [28]:
disease_list = ['AMD', 'Glaucoma','Glaucoma_suspect', 'DMR', 'Pathologic myopia', 'ERM', 'RVO']#, 'Other',]# 'Normal', 'Cataract'
color_list = ["#FF1818",'#F76E11', '#FFC600', '#519259', '#4D77FF', '#22577E', '#A63EC5', '#F473B9'] # "#FBF1D3",

In [6]:
model_name = 'resnet50'
image_size = 512
type_ = "ImageNet+Fundus"

In [7]:
# model = ResNetModel(model_name, 2)

# weight_path = f"../weights/{model_name}_{image_size}_{type_}.pth"
weight_path = "../fine_tuned_best_model_1.pt"

model = torch.load(weight_path, map_location="cpu")
model


/tmp/ipykernel_2944378/2255258716.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(weight_path, map_location="cpu")


OrderedDict([('conv1.weight',
              tensor([[[[ 0.0732,  0.0866,  0.0602,  ...,  0.0385,  0.0383,  0.0127],
                        [-0.2034, -0.1069, -0.0979,  ..., -0.1079, -0.1441, -0.2742],
                        [-0.0939, -0.0878, -0.0952,  ..., -0.0078, -0.0483, -0.0604],
                        ...,
                        [-0.3156, -0.0864, -0.1242,  ..., -0.2400, -0.1893, -0.1053],
                        [-0.3095, -0.2656, -0.2517,  ..., -0.0891, -0.1468, -0.1289],
                        [-0.3935, -0.3463, -0.3320,  ..., -0.0784, -0.0914, -0.1125]],
              
                       [[-0.1292,  0.1060, -0.0962,  ...,  0.1456,  0.1301,  0.0322],
                        [-0.1486, -0.1417, -0.1106,  ..., -0.0037, -0.0315, -0.1188],
                        [-0.1645, -0.1971, -0.2156,  ..., -0.0045,  0.0298,  0.0040],
                        ...,
                        [-0.2203, -0.2630, -0.3673,  ..., -0.6719, -0.5951, -0.4304],
                        [-0.1946, -0

In [8]:
import torchvision.transforms as transforms

In [7]:
# Load the checkpoint
checkpoint = torch.load(weight_path, map_location="cpu")

# If the model was wrapped in DataParallel, extract the state_dict
if isinstance(checkpoint, torch.nn.DataParallel):
    checkpoint = checkpoint.module.state_dict()
elif "state_dict" in checkpoint:
    checkpoint = checkpoint["state_dict"]

# Now load the state_dict into your model
model.load_state_dict(checkpoint, strict=False)  # You can use strict=False if necessary to handle mismatched keys
model.eval()

# Define image transformations (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

/tmp/ipykernel_2819881/1106536702.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weight_path, map_location="cpu")


AttributeError: 'collections.OrderedDict' object has no attribute 'load_state_dict'

In [10]:
# Initialize your model architecture
model = ResNetModel(model_name, 2)  # Replace this with your actual model initialization

# Load the checkpoint containing the state_dict
checkpoint = torch.load(weight_path, map_location="cpu")

# Check if the checkpoint is a DataParallel model and extract the state_dict
if isinstance(checkpoint, torch.nn.DataParallel):
    checkpoint = checkpoint.module.state_dict()
elif "state_dict" in checkpoint:
    checkpoint = checkpoint["state_dict"]

# Load the state_dict into your model
model.load_state_dict(checkpoint, strict=False)  # Set strict=False to handle missing or extra keys if needed
model.eval()

# Define image transformations (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


/tmp/ipykernel_2944378/3544424942.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weight_path, map_location="cpu")


In [11]:
# Function to predict glaucoma or non-glaucoma from an image
def predict_image(image_path, model):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        print("predicted.item()",predicted.item())
        class_name = 'glaucoma' if predicted.item() == 1 else 'non-glaucoma'

    return class_name


In [14]:
image_path = "../datasets/train/glaucoma/087.jpg"
prediction = predict_image(image_path, model)
print(f"The predicted label is: {prediction}")

predicted.item() 0
The predicted label is: non-glaucoma
